# 證券劃撥存款餘額  

- [台灣-證券劃撥存款餘額vs.台股](https://www.macromicro.me/charts/21957/tw-outstanding-securities-transfer-deposit-vs-tw-stock)
- [證券劃撥存款](https://www.cbc.gov.tw/tw/np-974-1.html)
- 檔案下載連結 [央行證券劃撥存款餘額](https://www.cbc.gov.tw/public/data/OpenData/%E7%B6%93%E7%A0%94%E8%99%95/%E8%AD%89%E5%88%B8%E5%8A%83%E6%92%A5%E4%B8%8A%E7%B6%B2%E8%B3%87%E6%96%99.csv)


In [1]:
import os
import sys
from pathlib import Path
from datetime import date
from datetime import datetime
import requests
import pandas as pd
import duckdb

In [2]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.common.duckdb_tool import insert_dataframe_to_duckdb

✅ 成功載入環境變數檔案: /Users/bryson0083/projects/TuaThanTsinn/src/TuaThanTsinn/proj_util_pkg/config/.env
[環境變數]PROJECT_ROOT: /Users/bryson0083/projects/TuaThanTsinn/src/TuaThanTsinn


## 公用參數設定

In [3]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [4]:
DOWNLOAD_FOLDER = os.environ.get('download_path')
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

f"下載資料存放路徑: {DOWNLOAD_FOLDER}"

'下載資料存放路徑: /Users/bryson0083/projects/TuaThanTsinn/src/TuaThanTsinn/temp'

In [5]:
# 新增偽裝成chrome瀏覽器的標頭
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

## 外部資料讀取  
### 方法1: 透過 open data web 取得csv檔案格式數據  
* 

In [6]:
TARGET_FILE_NAME = "證券劃撥上網資料.csv"

In [7]:
# 透過網址下載資料來源csv檔
data_url = "https://www.cbc.gov.tw/public/data/OpenData/%E7%B6%93%E7%A0%94%E8%99%95/%E8%AD%89%E5%88%B8%E5%8A%83%E6%92%A5%E4%B8%8A%E7%B6%B2%E8%B3%87%E6%96%99.csv"

# 設定下載檔案存放路徑
data_path = os.path.join(DOWNLOAD_FOLDER, TARGET_FILE_NAME)

# 透過Requests下載資料來源csv檔
response = requests.get(data_url)
with open(data_path, 'wb') as f:
    f.write(response.content)

In [8]:
# 讀取csv檔案內容
rep_path = os.path.join(DOWNLOAD_FOLDER, TARGET_FILE_NAME)
deposit_df = pd.read_csv(rep_path, encoding='utf-8', dtype=str)

print(deposit_df.shape)
deposit_df.head()

(419, 9)


,年月,證券劃撥存款月底餘額億元,證券劃撥存款月底增減金額億元,證券劃撥存款旬平均餘額億元,證券劃撥存款旬平均增減金額億元,證券劃撥存款對M1B年增率之貢獻率%,股票市場融資金額月底餘額億元,股票市場成交值日平均億元,股票市場股價指數月平均
0,80/01,"1,837",0,0,0,0.00,379,354,"3,938"
1,80/02,"1,910",73,"1,874",0,0.00,521,570,"4,643"
2,80/03,"1,809",-101,"1,860",-14,0.00,562,481,"4,781"
3,80/04,"1,945",136,"1,877",18,0.00,708,750,"5,606"
4,80/05,"1,743",-202,"1,844",-33,0.00,725,611,"5,993"


In [9]:
# 將民國年月格式轉換為西元年月格式
# 例如: "80/05" -> "199105"
def convert_roc_to_ad(roc_date):
    """
    將民國年月轉換為西元年月
    Input: "80/05" (民國80年5月)
    Output: "199105" (1991年5月)
    """
    year, month = roc_date.split('/')
    ad_year = int(year) + 1911  # 民國年 + 1911 = 西元年
    return f"{ad_year}{month.zfill(2)}"  # 確保月份是兩位數

# 轉換年月欄位
deposit_df['年月'] = deposit_df['年月'].apply(convert_roc_to_ad)

# 將年月欄位重新命名為 Date
deposit_df.rename(columns={'年月': 'Date'}, inplace=True)

# 將 Date 欄位移到第一個位置
cols = deposit_df.columns.tolist()
cols.remove('Date')
deposit_df = deposit_df[['Date'] + cols]

print("轉換後的年月格式並調整欄位順序:")
deposit_df.head()

轉換後的年月格式並調整欄位順序:


,Date,證券劃撥存款月底餘額億元,證券劃撥存款月底增減金額億元,證券劃撥存款旬平均餘額億元,證券劃撥存款旬平均增減金額億元,證券劃撥存款對M1B年增率之貢獻率%,股票市場融資金額月底餘額億元,股票市場成交值日平均億元,股票市場股價指數月平均
0,199101,"1,837",0,0,0,0.00,379,354,"3,938"
1,199102,"1,910",73,"1,874",0,0.00,521,570,"4,643"
2,199103,"1,809",-101,"1,860",-14,0.00,562,481,"4,781"
3,199104,"1,945",136,"1,877",18,0.00,708,750,"5,606"
4,199105,"1,743",-202,"1,844",-33,0.00,725,611,"5,993"


## 資料留存ＤＢ

In [10]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [11]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

### 資料留存

In [12]:
table_name = "tw_securities_giro_deposit"

In [13]:
insert_row_count = insert_dataframe_to_duckdb(
    conn_duckdb, 
    deposit_df, 
    table_name, 
    date_column='Date'
)

f"成功插入 {insert_row_count} 筆資料到 {table_name}"

INFO:proj_util_pkg.common.duckdb_tool:成功插入 0 筆新資料到 tw_securities_giro_deposit


'成功插入 0 筆資料到 tw_securities_giro_deposit'

In [14]:
# 確認寫入結果
conn_duckdb.execute(f"SELECT * FROM {table_name} order by Date desc LIMIT 10").fetch_df()

,Date,證券劃撥存款月底餘額億元,證券劃撥存款月底增減金額億元,證券劃撥存款旬平均餘額億元,證券劃撥存款旬平均增減金額億元,證券劃撥存款對M1B年增率之貢獻率%,股票市場融資金額月底餘額億元,股票市場成交值日平均億元,股票市場股價指數月平均,create_datetime
0,202511,"36,068","-2,629","36,952","-1,981",1.11,"4,177","5,316","27,464",2026-01-10 10:28:09.772
1,202510,"38,697",-260,"38,933",407,1.47,"4,103","5,030","27,401",2026-01-10 10:28:09.772
2,202509,"38,957",927,"38,526",702,1.55,"3,844","4,523","25,222",2026-01-10 10:28:09.772
3,202508,"38,030","1,322","37,824","2,176",1.13,"3,605","4,197","24,045",2026-01-10 10:28:09.772
4,202507,"36,709","2,635","35,648","1,903",0.18,"3,152","3,199","22,963",2026-01-10 10:28:09.772
5,202506,"34,073","1,071","33,745",744,-0.49,"3,054","3,430","22,014",2026-01-10 10:28:09.772
6,202505,"33,002",762,"33,001",635,-0.76,"2,977","3,108","21,271",2026-01-10 10:28:09.772
7,202504,"32,240",141,"32,366",-79,-0.73,"2,818","2,711","19,558",2026-01-10 10:28:09.772
8,202503,"32,099","-1,446","32,445","-1,014",-0.92,"4,256","3,040","22,194",2026-01-10 10:28:09.772
9,202502,"33,545",712,"33,459",411,-0.12,"4,401","3,626","23,328",2026-01-10 10:28:09.772


In [15]:
# 關閉資料庫連線
conn_duckdb.close()

In [16]:
# 清除下載暫存檔案
os.remove(data_path)